In [5]:
import cv2
import numpy as np
import time

import mediapipe as mp
mp_objectron = mp.solutions.objectron
mp_drawing = mp.solutions.drawing_utils


#shoe_images = {'image': cv2.imread('shoe.png')}
#cv2.imshow('image', shoe_images['image'])
vid = cv2.VideoCapture(0)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(5)



while True:

  _, img = vid.read()
  shoe_images = {'image': img}
  with mp_objectron.Objectron(
      static_image_mode=True,
      max_num_objects=5,
      min_detection_confidence=0.5,
      model_name='Shoe') as objectron:
    # Run inference on shoe images.
    for name, image in shoe_images.items():
      # Convert the BGR image to RGB and process it with MediaPipe Objectron.
      results = objectron.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

      # Draw box landmarks.
      if not results.detected_objects:
        print(f'No box landmarks detected on {name}')
        continue
      print(f'Box landmarks of {name}:')
      annotated_image = image.copy()
      for detected_object in results.detected_objects:
        mp_drawing.draw_landmarks( annotated_image, detected_object.landmarks_2d, mp_objectron.BOX_CONNECTIONS)
        mp_drawing.draw_axis(annotated_image, detected_object.rotation, detected_object.translation)
      cv2.imshow('img', annotated_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
cv2.waitKey(0)
cv2.destroyAllWindows()

No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks detected on image
No box landmarks det

In [1]:
import cv2
import time

vid = cv2.VideoCapture(0)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(5)


while True:
    _, img = vid.read()
    cv2.imshow('image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    #cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
import cv2
import numpy as np
import time

import mediapipe as mp

img = cv2.imread('shoe.png')
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

mp_objectron = mp.solutions.objectron
mp_drawing = mp.solutions.drawing_utils

objectron = mp_objectron.Objectron(mp_objectron.Objectron(static_image_mode=True, max_num_objects=5, min_detection_confidence=0.5, model_name='Shoe'))
results = objectron.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ann_img = img.copy()
detected_object = results.detected_objects[0]
mp_drawing.draw_landmarks( ann_img, detected_object.landmarks_2d, mp_objectron.BOX_CONNECTIONS)
mp_drawing.draw_axis( ann_img, detected_object.rotation, detected_object.translation)
cv2.imshow( 'ann_img', ann_img)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [10]:
results.detected_objects[0].rotation

array([[ 0.9117344 ,  0.06848495,  0.930058  ],
       [ 0.27734783,  0.98235393, -0.27088964],
       [-0.30301553,  0.17404222,  0.24821527]])

In [1]:
from osgeo import gdal



ModuleNotFoundError: No module named '_gdal'

In [1]:
from PIL import Image

img = Image.open('/home/hameed/s2p/tests/testoutput/output_pair/tiles/row_0003000_height_1000/col_0003000_width_1000/pair_1/rectified_sec.tif')
img.show()

UnidentifiedImageError: cannot identify image file '/home/hameed/s2p/tests/testoutput/output_pair/tiles/row_0003000_height_1000/col_0003000_width_1000/pair_1/rectified_sec.tif'